In [1]:
import pandas as pd

In [2]:
db=pd.read_csv('adult.csv')

In [3]:
db

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [4]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


удалям ?,  их немного (вывод из предыдущего дз)

In [5]:
for i in range(len(db['age'])):
    if '?' in db.loc[i].values.tolist():
        db=db.drop(i, axis=0)

In [6]:
#кодируем таргет, как 1 и 0

In [7]:
income_res=[]
for i in db['income'].values.tolist():
    if i=='<=50K':
        income_res.append(1)
    else:
        income_res.append(0)
db['income_target']=income_res
db=db.drop('income', axis=1)

In [8]:
#делим на тест и трейн

In [9]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(db, test_size=0.2, random_state=42)

In [10]:
#находим категориальные переменные

In [11]:
import numpy as np
res2=[]
for i in db.columns:
    res1=[]
    for j in db[i].values.tolist():
        if isinstance(j, str)==True:
            res1.append(1)
        else:
            res1.append(0)
    if sum(res1)>0:
        res2.append(i)
    else:
        res2.append(0)
print (res2)

[0, 'workclass', 0, 'education', 0, 'marital-status', 'occupation', 'relationship', 'race', 'gender', 0, 0, 0, 'native-country', 0]


In [12]:
category_columns=[x for x in res2 if x!=0]

#для порядковых переменных можно использовать labelEncoder (worlclass, education, marital-status, relationship), для остальных woe, но для скорости всех закодируем как woe

In [13]:
def get_woe_v1(db, db_test, col, target_col):
    all_good = len(db[db[target_col] == 1][col])
    all_bad = len(db[db[target_col] == 0][col])
    odds_series = (
        db[db[target_col] == 1][col].value_counts()
        /
        db[db[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    db[col + '_woe'] = db[col].apply(category_woe_dict.get)
    db_test[col + '_woe'] = db_test[col].apply(category_woe_dict.get)
    return db, db_test

In [14]:
for i in category_columns:
    get_woe_v1(train_set, test_set ,i, 'income_target')

C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [15]:
train_set=train_set.drop(category_columns, axis=1)
test_set=test_set.drop(category_columns, axis=1)

In [16]:
corr_matrix=train_set.corr()

In [17]:
corr_matrix

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income_target,workclass_woe,education_woe,marital-status_woe,occupation_woe,relationship_woe,race_woe,gender_woe,native-country_woe
age,1.000000,-0.074184,0.039578,0.079746,0.059814,0.103438,-0.239593,-0.184865,-0.086794,-0.385446,-0.144186,-0.408481,-0.026356,-0.081418,-0.060128
fnlwgt,-0.074184,1.000000,-0.041268,-0.000201,-0.006288,-0.021149,0.005027,0.037004,0.037328,0.034522,0.010689,0.032467,0.103301,-0.027418,0.120438
educational-num,0.039578,-0.041268,1.000000,0.127142,0.081148,0.149340,-0.333502,-0.150889,-0.966184,-0.073788,-0.454160,-0.122541,-0.092575,-0.005935,-0.253661
capital-gain,0.079746,-0.000201,0.127142,1.000000,-0.032315,0.083739,-0.221154,-0.090947,-0.148052,-0.081119,-0.098178,-0.083861,-0.026603,-0.051509,-0.018453
capital-loss,0.059814,-0.006288,0.081148,-0.032315,1.000000,0.051973,-0.148969,-0.043774,-0.089599,-0.074451,-0.081657,-0.078864,-0.024563,-0.044163,-0.022906
hours-per-week,0.103438,-0.021149,0.149340,0.083739,0.051973,1.000000,-0.230053,-0.129021,-0.170677,-0.240760,-0.215435,-0.289865,-0.049973,-0.232458,-0.019033
income_target,-0.239593,0.005027,-0.333502,-0.221154,-0.148969,-0.230053,1.000000,0.165910,0.352802,0.444984,0.335405,0.435619,0.097773,0.217528,0.095822
workclass_woe,-0.184865,0.037004,-0.150889,-0.090947,-0.043774,-0.129021,0.165910,1.000000,0.163468,0.143794,0.181977,0.146810,0.021860,0.087234,0.054968
education_woe,-0.086794,0.037328,-0.966184,-0.148052,-0.089599,-0.170677,0.352802,0.163468,1.000000,0.100751,0.463618,0.154485,0.098757,0.021109,0.204705
marital-status_woe,-0.385446,0.034522,-0.073788,-0.081119,-0.074451,-0.240760,0.444984,0.143794,0.100751,1.000000,0.214432,0.892875,0.123564,0.400468,0.030330


In [18]:
train_set=train_set.drop('educational-num', axis=1)
test_set=test_set.drop('educational-num', axis=1)

In [19]:
corr_target=corr_matrix['income_target'].abs().sort_values()
corr_target_list=corr_target.index.tolist()
corr_target_list.remove('marital-status_woe')
corr_target_list.remove('educational-num')
corr_target_list.remove('income_target')
train_set=train_set.dropna()

In [35]:
test_set=test_set.dropna()

In [36]:
X=train_set.drop('income_target', axis=1)
Y=train_set['income_target']
X_test=test_set.drop('income_target', axis=1)
Y_test=test_set['income_target']

In [21]:
from sklearn.model_selection import GridSearchCV


In [33]:
from sklearn import  svm
model1 = svm.SVC()
model1.fit(X,Y)
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
parameters = {'max_depth':[2,3,4,5,6,7,8,9,10],'max_features':[0.33, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
model_best = GridSearchCV(model, parameters, cv=10, n_jobs=-1, scoring='roc_auc')
model_best.fit(X, Y)
best=model_best.best_params_
model2=DecisionTreeClassifier(max_depth=best['max_depth'], max_features=best['max_features'])
model2.fit(X,Y)
from sklearn.neighbors import KNeighborsClassifier
model3= KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
model3.fit(X,Y)
model4=KNeighborsClassifier(n_neighbors=5, n_jobs=-1, algorithm='kd_tree')
model4.fit(X,Y)
model5=KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
model5.fit(X,Y)

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='uniform')

In [37]:
from sklearn.metrics import roc_auc_score
ypred1=model1.predict(X_test)
print(roc_auc_score(Y_test, ypred1))
ypred2=model2.predict(X_test)
print(roc_auc_score(Y_test, ypred2))
ypred3=model3.predict(X_test)
print(roc_auc_score(Y_test, ypred3))
ypred4=model4.predict(X_test)
print(roc_auc_score(Y_test, ypred4))
ypred5=model5.predict(X_test)
print(roc_auc_score(Y_test, ypred5))

0.5107039330872996
0.7732791248528712
0.629441428802963
0.629441428802963
0.6220056141413658


In [38]:
model_best.cv_results_

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\volko\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\volko\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\volko\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([0.0480324 , 0.0526423 , 0.05778024, 0.06331048, 0.06995845,
        0.0748414 , 0.07890553, 0.060129  , 0.06534956, 0.07462318,
        0.07608268, 0.08803697, 0.09837096, 0.10629444, 0.06770134,
        0.08094187, 0.07699535, 0.08535726, 0.1057621 , 0.11672871,
        0.12007694, 0.07430789, 0.08710847, 0.09676085, 0.11484077,
        0.1281167 , 0.13847547, 0.15092714, 0.08263533, 0.10089364,
        0.11487744, 0.11783824, 0.14720647, 0.1579179 , 0.17563713,
        0.09082894, 0.10610919, 0.12432525, 0.13305001, 0.15510869,
        0.1653394 , 0.17522597, 0.10075347, 0.10948503, 0.13399377,
        0.15680139, 0.18721426, 0.20664992, 0.21406271, 0.10981123,
        0.13144758, 0.1504009 , 0.16634281, 0.19175498, 0.20705092,
        0.21803534, 0.11348307, 0.13548889, 0.1574007 , 0.17831306,
        0.2111455 , 0.23319056, 0.24907255]),
 'std_fit_time': array([0.00718507, 0.00636533, 0.0101945 , 0.00873045, 0.00635733,
        0.00606917, 0.00823905, 0.005